In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('Test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/22 03:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path = 'airtrafficdata/airtraffic_all/airtraffic-part/flightmonth=200801/part-00252-5cde1303-4ebf-4a12-8fad-f5d9f9c9124a-Copy1.c000.snappy.parquet'

In [3]:
airtraffic = spark.read.parquet(path)

In [4]:
airtraffic.count()

605659

In [5]:
airtraffic.select(count(lit(1)).alias('count')).show()

+------+
| count|
+------+
|605659|
+------+



In [6]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    describe(). \
    show()

+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|605659|605659|            605659|
|   mean|2008.0|   1.0|15.908469947610785|
| stddev|   0.0|   0.0| 8.994294747375292|
|    min|  2008|     1|                 1|
|    max|  2008|     1|                31|
+-------+------+------+------------------+



In [7]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    summary(). \
    show()

23/08/22 03:49:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|605659|605659|            605659|
|   mean|2008.0|   1.0|15.908469947610785|
| stddev|   0.0|   0.0| 8.994294747375292|
|    min|  2008|     1|                 1|
|    25%|  2008|     1|                 8|
|    50%|  2008|     1|                16|
|    75%|  2008|     1|                24|
|    max|  2008|     1|                31|
+-------+------+------+------------------+



In [8]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    distinct(). \
    count()

31

In [9]:
airtraffic.select(countDistinct('Year','Month','DayOfMonth').alias('count distinct')).show()

+--------------+
|count distinct|
+--------------+
|            31|
+--------------+



In [10]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [11]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [12]:
employeesDF.show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [13]:
employeesDF. \
    select(((sum(coalesce(col('bonus').cast('int'),lit(0))*col('salary'))) / lit(100)).alias('Total Bonus')).show()

+-----------+
|Total Bonus|
+-----------+
|      250.0|
+-----------+



In [14]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [15]:
airtraffic.select('IsDepDelayed').distinct().show()

+------------+
|IsDepDelayed|
+------------+
|         YES|
|          NO|
+------------+



In [16]:
airtraffic. \
    filter(col('IsDepDelayed') == 'YES'). \
    select(sum(col('DepDelay')).alias('total dep delay')).show()

+---------------+
|total dep delay|
+---------------+
|      8160339.0|
+---------------+

